# Trading Agent using Bayesian Networks

This notebook demonstrates how to build a trading agent using Bayesian Networks. It fetches stock market data, processes it, and uses Bayesian inference for decision-making.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pgmpy
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

## Data Collection

In [ ]:
ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end='2023-12-31')
data.to_csv(f'{ticker}_data.csv')

## Bayesian Network

In [ ]:
# Define the network structure
model = BayesianNetwork([
    ('RSI_Cat', 'Price_Movement'),
    ('Vol_Cat', 'Price_Movement'),
    ('Vol_Chg', 'Price_Movement'),
    ('Price_Movement', 'Action')
])

# Prepare data columns (dummy placeholders for RSI_Cat, Vol_Cat, etc.)
# Make sure you have RSI_Cat, Vol_Cat, Vol_Chg, Price_Movement columns in your data.
# Below is simplified demonstration code.
data['Price_Movement'] = pd.to_numeric(data['Close'].pct_change(), errors='coerce')
data['Action'] = data['Price_Movement'].apply(lambda x: 'Buy' if x > 0 else 'Sell' if x < 0 else 'Hold')

# Split into train/test
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# Create categorical placeholders
train_data['RSI_Cat'] = pd.cut(np.random.rand(len(train_data)), bins=[0, 0.3, 0.7, 1.0], labels=['Low', 'Medium', 'High'])
train_data['Vol_Cat'] = pd.cut(np.random.rand(len(train_data)), bins=[0, 0.3, 0.7, 1.0], labels=['Low', 'Medium', 'High'])
train_data['Vol_Chg'] = np.sign(np.random.randn(len(train_data)))
train_data['Vol_Chg'] = train_data['Vol_Chg'].replace({1: 'Up', -1: 'Down', 0: 'No Change'})

train_data['Price_Movement'] = np.sign(train_data['Price_Movement'])
train_data['Price_Movement'] = train_data['Price_Movement'].replace({1: 'Up', -1: 'Down', 0: 'No Change'})

# We'll keep only the columns needed
train_data = train_data[['RSI_Cat', 'Vol_Cat', 'Vol_Chg', 'Price_Movement', 'Action']].dropna()

# Fit the model
model.fit(train_data, estimator=MaximumLikelihoodEstimator)
# Prepare inference object
inference = VariableElimination(model)

## Evaluation

In [ ]:
def calculate_returns(model, test_data):
    portfolio = 10000.0
    holdings = 0.0

    # For simplicity, create the same categorical placeholders for test_data
    test_data = test_data.copy()
    test_data['RSI_Cat'] = pd.cut(np.random.rand(len(test_data)), bins=[0, 0.3, 0.7, 1.0], labels=['Low', 'Medium', 'High'])
    test_data['Vol_Cat'] = pd.cut(np.random.rand(len(test_data)), bins=[0, 0.3, 0.7, 1.0], labels=['Low', 'Medium', 'High'])
    test_data['Vol_Chg'] = np.sign(np.random.randn(len(test_data)))
    test_data['Vol_Chg'] = test_data['Vol_Chg'].replace({1: 'Up', -1: 'Down', 0: 'No Change'})
    # Price movement sign
    test_data['Price_Movement'] = np.sign(test_data['Close'].pct_change()).replace({1: 'Up', -1: 'Down', 0: 'No Change'})
    # For consistent labeling with model training:
    test_data['Action'] = test_data['Price_Movement'].apply(
        lambda x: 'Buy' if x == 'Up' else 'Sell' if x == 'Down' else 'Hold'
    )

    # Traverse test data and apply the model's decisions
    for idx, row in test_data.iterrows():
        # If needed, we create an inference query.
        # The model's structure used RSI_Cat, Vol_Cat, Vol_Chg -> Price_Movement -> Action.
        # So we query for 'Action' given RSI_Cat, Vol_Cat, Vol_Chg.
        if pd.isna(row['RSI_Cat']) or pd.isna(row['Vol_Cat']) or pd.isna(row['Vol_Chg']):
            continue  # skip rows with missing categories

        evidence = {
            'RSI_Cat': row['RSI_Cat'],
            'Vol_Cat': row['Vol_Cat'],
            'Vol_Chg': row['Vol_Chg']
        }
        # MAP query (most probable action)
        action = inference.map_query(variables=['Action'], evidence=evidence)['Action']

        # Simple logic: if action is Buy, invest all; if Sell, liquidate all.
        price = row['Close']
        if price is not None and not np.isnan(price):
            if action == 'Buy' and portfolio > 0:
                holdings += portfolio / price
                portfolio = 0
            elif action == 'Sell' and holdings > 0:
                portfolio += holdings * price
                holdings = 0

    # Final value if we still have holdings at the end
    if holdings > 0 and len(test_data) > 0:
        final_price = test_data.iloc[-1]['Close']
        portfolio += holdings * final_price
        holdings = 0

    return portfolio

# Calculate returns from the model
model_returns = calculate_returns(model, test_data)

# Calculate naive buy & hold returns
initial_price = test_data.iloc[0]['Close'] if len(test_data) > 0 else 1
final_price = test_data.iloc[-1]['Close'] if len(test_data) > 0 else 1
buy_hold_returns = 10000.0 * (final_price / initial_price)

print(f"Model Returns: ${model_returns:.2f}")
print(f"Buy & Hold Returns: ${buy_hold_returns:.2f}")